In [1]:
%matplotlib inline

#### Load constants ####
import scipy.constants as sc

from numpy import *

import matplotlib
from matplotlib.pyplot import *
from matplotlib.colors import LogNorm
import glob
import os

import sys

import adios2

# OVERWRITE DEFAULT PLOTTING PARAMETERS
params = {
    'font.family' : 'sans-serif',
    'font.sans-serif' : ['DejaVu Sans', 'Arial', 'Verdana', 'Liberation Sans'],
    'mathtext.default' : 'regular',
    'mathtext.rm' : 'sans',
    'font.size' : 18,
    'legend.fontsize' : 14,
    'axes.labelsize' : 18,
    'axes.titlesize' : 16,
    'lines.linewidth' : 3.0,
    'legend.frameon' : False,
    'legend.numpoints': 1,
}

matplotlib.rcParams.update(params)

In [2]:
files = ["/home/marqua27/jupyter_notebooks/diags/test_gauss_00000.bp"]

#files = glob.glob(path + "/openPMD/" + "*.bp5")

def extractStepFunc(filename):
    return int((filename.rpartition('_')[2]).rpartition('.')[0])

files.sort(key=extractStepFunc)#(key=os.path.getmtime)

#files.sort(key=os.path.getmtime)

print("Number of files = %i"%(len(files)))

#print(files)

Number of files = 1


# Grid parameters

In [ ]:
with adios2.Stream(files[0], "r") as f:
    for _ in f.steps():

        t_i = extractStepFunc(files[0])
        
        print("index order:", f.read_attribute("/data/%i/fields/B/axisLabels"%(t_i)))

        Nz, Ny, Nx = array(f.available_variables().get("/data/%i/fields/B/x"%(t_i))['Shape'].split(', '), dtype=int)
        print("Nz = %i, Ny = %i, Nx = %i"%(Nz,Ny,Nx))

        unit_length_SI = f.read_attribute("/data/%i/unit_length"%(t_i))
        Dy_SI = f.read_attribute("/data/%i/cell_height"%(t_i)) * unit_length_SI
        Dx_SI = f.read_attribute("/data/%i/cell_width"%(t_i)) * unit_length_SI
        Dz_SI = f.read_attribute("/data/%i/cell_depth"%(t_i)) * unit_length_SI

        unit_time_SI = f.read_attribute("/data/%i/unit_time"%(t_i))
        Dt_SI = f.read_attribute("/data/%i/dt"%(t_i)) * unit_time_SI
        print("Δt = %es"%(Dt_SI))
        print("Simulation volume: %.2f x %.2f x %.2f nm^3"%(Nx*Dx_SI*1.e9, Ny*Dy_SI*1.e9, Nz*Dz_SI*1.e9))

laser_peak_at_target = 89603 # FWHM_I=40fs ##37487 # FWHM_I=16fs

# Read fields

In [ ]:
for i in arange(len(files)): #array([-1]): #arange(len(files)): #
    with adios2.Stream(files[i], "r") as f:
        for _ in f.steps():
            
            t_i = extractStepFunc(files[i])
            #print(f"Current step is {t_i}")
            """"""
            unit_fieldE = f.read_attribute("/data/%i/fields/E/x/unitSI"%(t_i))
            Ex = f.read("/data/%i/fields/E/x"%(t_i), start=[int(Nz//2),0,0], count=[1,Ny,Nx]) * unit_fieldE
            Ey = f.read("/data/%i/fields/E/y"%(t_i), start=[int(Nz//2),0,0], count=[1,Ny,Nx]) * unit_fieldE
            Ez = f.read("/data/%i/fields/E/z"%(t_i), start=[int(Nz//2),0,0], count=[1,Ny,Nx]) * unit_fieldE
            #Ex = f.read("/data/%i/fields/E/x"%(t_i)) #* unit_fieldE
            #Ey = f.read("/data/%i/fields/E/y"%(t_i)) #* unit_fieldE
            #Ez = f.read("/data/%i/fields/E/z"%(t_i)) #* unit_fieldE
            """"""
            """"""
            unit_fieldB = f.read_attribute("/data/%i/fields/B/z/unitSI"%(t_i))
            Bx = f.read("/data/%i/fields/B/x"%(t_i), start=[int(Nz//2),0,0], count=[1,Ny,Nx]) * unit_fieldB
            By = f.read("/data/%i/fields/B/y"%(t_i), start=[int(Nz//2),0,0], count=[1,Ny,Nx]) * unit_fieldB
            Bz = f.read("/data/%i/fields/B/z"%(t_i), start=[int(Nz//2),0,0], count=[1,Ny,Nx]) * unit_fieldB
            #Bx = f.read("/data/%i/fields/B/x"%(t_i)) #* unit_fieldB
            #By = f.read("/data/%i/fields/B/y"%(t_i)) #* unit_fieldB
            #Bz = f.read("/data/%i/fields/B/z"%(t_i)) #* unit_fieldB
            """"""
            ## charge density electrons
            unit_fieldRho_e = f.read_attribute("/data/%i/fields/He_e_all_density/unitSI"%(t_i))
            #rho_e = f.read("/data/%i/fields/He_e_all_density"%(t_i)) #* unit_fieldRho_e
            rho_e = f.read("/data/%i/fields/He_e_all_density"%(t_i), start=[0,int(300.e-9/Dy_SI),0], count=[Nz,1,Nx]) * unit_fieldRho_e
            ## charge density ions
            #unit_fieldRho_i = f.read_attribute("/data/%i/fields/He_i_all_density/unitSI"%(t_i))
            #rho_i = f.read("/data/%i/fields/He_i_all_density"%(t_i)) #* unit_fieldRho_i
            #rho_i = f.read("/data/%i/fields/He_i_all_density"%(t_i), start=[int(Nz//2),0,0], count=[1,Ny,Nx]) #* unit_fieldRho_i
    
    #################
    ##  Plot data  ##
    #################
    
    # instantaneous energy density = 0.5*eps0*(||E||**2 + c**2 * ||B||**2)
    # time averaged energy density is approximately 0.5 of maximum of instantaneous energy density due to average(cos**2) = 1/2 with which the E- and B-field oscillates
    #energy_dens_avg = 0.5*sc.epsilon_0*0.5*((Ez**2 + Ey**2 + Ex**2)*unit_fieldE**2 + sc.c*(Bz**2 + By**2 + Bx**2)*unit_fieldB**2)
    #plot_field = energy_dens_avg[0,:,:] * sc.c*1.e-4 / I0
    #plot_field = average(energy_dens_avg, axis=0)*sc.c*1.e-4 / I0 # sum over transverse axis z (laser propagates along y)
    #plot_field = plot_field[Nz//2, :, :] # take the central y-x slice
    plot_field_norm = sqrt(2*I0*1.e4/(sc.c*sc.epsilon_0))
    plot_field = Ex[0,:,:] / plot_field_norm
    
    #print("Maximum Intensity %.3eW/cm^2"%(sc.c*energy_dens_avg.max()*1.e-4))
    print("Maximum E-field = %.3eV/m"%(sqrt(Ex**2+Ey**2+Ez**2).max()*unit_fieldE))
    figure(figsize=(24,6))
    
    ##
    ## Plot 1
    ##
    subplot(1,2,1)
    imshow(plot_field
        , matplotlib.cm.RdBu_r
        #, mpl.cm.cubehelix_r
        , origin='lower', interpolation='nearest', aspect='equal'
        , extent=(0, Nx*Dx_SI*1.e9, 0, Ny*Dy_SI*1.e9)
        #, norm=LogNorm()#vmin = 1.e-6, vmax = 1.5)
        , vmin = -2., vmax = 2.
    )
    
    colorbar()
    
    title("field energy density integrated\nalong z @t=%ffs after peak"%((t_i-laser_peak_at_target)*Dt_SI*1.e15))
    
    xlabel("x [nm]")
    ylabel("y [nm]")
    
    
    ##
    ## Plot 2
    ##
    subplot(1,2,2)
    imshow(rho_e[:, 0, :] / rho_e.max() #sum(rho_e, axis=1)
        , matplotlib.cm.plasma
        , origin='lower', interpolation='nearest', aspect='equal'
        , extent=(0, Nx*Dx_SI*1.e9, 0, Nz*Dz_SI*1.e9)
        , norm=LogNorm(vmin = 1.e-6, vmax = 1.)
        #, vmin = 0., vmax = .3
    )
    
    colorbar()
    
    title("e^- number density\nin plane @t=%iΔt"%(t_i))
    
    xlabel("x [nm]")
    #xticks(array([0,50,100,150,200,250]))
    ylabel("z [nm]")


    ##
    ## Plot 3
    ##
    """
    subplot(1,3,3)
    imshow(rho_i[0,:,:] #sum(rho_i, axis=1)
        , matplotlib.cm.plasma
        , origin='lower', interpolation='nearest', aspect='equal'
        , extent=(0, Nx*Dx_SI*1.e9, 0, Nz*Dz_SI*1.e9)
        #, norm=LogNorm(vmin = 1.e-4, vmax = .3)
        #, vmin = 0., vmax = .3
    )
    
    colorbar()
    
    title("ion number density\nin plane @t=%iΔt"%(t_i))
    
    xlabel("x [nm]")
    ylabel("z [nm]")
    """
    
    #savefig("")
    
    tight_layout()
    
    show()
    close()


In [ ]:
print(plot_field.max())

# Read particle data

In [ ]:
for i in arange(len(files)): #array([-1]): #
    with adios2.Stream(files[i], "r") as f:
        for _ in f.steps():
            
            t_i = extractStepFunc(files[i])

            #He_i_position_y = f.read("/data/%i/particles/He_i/positionOffset/y"%(t_i))
            unit_eMom = f.read_attribute("/data/%i/particles/He_e/momentum/x/unitSI"%(t_i))
            He_e_mom_x = f.read("/data/%i/particles/He_e/momentum/x"%(t_i)) * unit_eMom
            He_e_mom_y = f.read("/data/%i/particles/He_e/momentum/y"%(t_i)) * unit_eMom
            He_e_mom_z = f.read("/data/%i/particles/He_e/momentum/z"%(t_i)) * unit_eMom
            
    
    mom_tot = sqrt(He_e_mom_x**2 + He_e_mom_y**2 + He_e_mom_z**2)
    avg_mom = average(mom_tot)
    avg_energy = 0.5*avg_mom**2/sc.m_e
    print("Electron temperature @t=%iΔt = %.3eeV"%(t_i, avg_energy/sc.e))
    """
    figure()
    hist(mom_tot, bins=128) #, range=(0,800))
    title("total mom_e (normalized) @t=%iΔt"%(t_i))
    grid()
    show()
    close()
    """

In [ ]:
BASE_DENSITY_SI = 1.71467764e+29
e_DENSITY_SI = BASE_DENSITY_SI #* 0.5
kBT_target = e_DENSITY_SI*sc.e*Dx_SI**2/sc.epsilon_0 # unit: eV
print("Target temperature to resolve debye length on grid k_BT = %fkeV"%(kBT_target*1.e-3))

In [ ]:
for i in array([0]): #arange(len(files)): #array([-1]): #
    with adios2.Stream(files[i], "r") as f:
        for _ in f.steps():
            
            t_i = extractStepFunc(files[i])

            He_i_position_y = f.read("/data/%i/particles/He_e/positionOffset/y"%(t_i))
    
    figure()
    hist(He_i_position_y, bins=200) #, range=(0,800))
    title("He_i position @t=%iΔt"%(t_i))
    grid()
    show()
    close()

In [ ]:
for i in array([0]): #arange(len(files)): #array([-1]): #
    with adios2.Stream(files[i], "r") as f:
        for _ in f.steps():
            
            t_i = extractStepFunc(files[i])

            He_i_weights = f.read("/data/%i/particles/He_i/weighting"%(t_i))
            He_e_weights = f.read("/data/%i/particles/He_e/weighting"%(t_i))
    
    print("Weighting ration He_e/He_i =", sum(He_e_weights)/sum(He_i_weights))

# ADIOS2 file content

In [3]:
with adios2.Stream(files[0], "r") as testf:
    for _ in testf.steps():

        testf_variables = testf.available_variables()
        print("VARIABLES:", len(testf_variables), '\n')

    #    #print(testf_variables)
    #    print("available values:", testf_variables.get("/data/97347/fields/B/x") , '\n')

        for key, val in testf_variables.items():
            print(key, ":", val['Shape'])

        print("="*60)


        testf_attributes = testf.available_attributes()
        print( "ATTRIBUTES:", len(testf_attributes), '\n' )

        print("available values:", testf_attributes.get("/author") , '\n')

        for key, val in testf_attributes.items():
            print("%s: %s (%s)"%(key, val['Value'], val['Type']))

        print("="*60)


VARIABLES: 1 

/data/0/meshes/laserEnvelope : 1, 800, 7759
ATTRIBUTES: 26 

available values: None 

/basePath: "/data/%T/" (string)
/data/0/dt: 1 (double)
/data/0/meshes/laserEnvelope/angularFrequency: 1.78715e+15 (double)
/data/0/meshes/laserEnvelope/axisLabels: { "t", "r" } (string)
/data/0/meshes/laserEnvelope/dataOrder: "C" (string)
/data/0/meshes/laserEnvelope/envelopeField: "normalized_vector_potential" (string)
/data/0/meshes/laserEnvelope/geometry: "thetaMode" (string)
/data/0/meshes/laserEnvelope/gridGlobalOffset: { -2.25e-13, 0 } (double)
/data/0/meshes/laserEnvelope/gridSpacing: { 4.50563e-16, 1.41789e-06 } (double)
/data/0/meshes/laserEnvelope/gridUnitSI: 1 (double)
/data/0/meshes/laserEnvelope/polarization: { 1, 0 } (double)
/data/0/meshes/laserEnvelope/position: { 0, 0 } (long double)
/data/0/meshes/laserEnvelope/timeOffset: 0 (float)
/data/0/meshes/laserEnvelope/unitDimension: { 0, 0, 0, 0, 0, 0, 0 } (double)
/data/0/meshes/laserEnvelope/unitSI: 1 (double)
/data/0/time: